# Carrier Agreements & Guides (Unstructured)
Parse PDFs with Azure Document Intelligence and write contract entities (rates, surcharges, clauses) into Lakehouse Delta tables.

In [ ]:
# Fabric notebook prerequisites
# - Attach to your Lakehouse
# - Set env vars via Fabric (Key Vault/runtime): FORMREC_ENDPOINT, FORMREC_KEY
import os
from datetime import datetime
from typing import List
import pandas as pd
from pyspark.sql import SparkSession

try:
    from azure.ai.formrecognizer import DocumentAnalysisClient
    from azure.core.credentials import AzureKeyCredential
except Exception as e:
    print('Install azure-ai-formrecognizer in this environment to run parsing.')
    raise

FORMREC_ENDPOINT = os.getenv('FORMREC_ENDPOINT')
FORMREC_KEY = os.getenv('FORMREC_KEY')
client = DocumentAnalysisClient(FORMREC_ENDPOINT, AzureKeyCredential(FORMREC_KEY))


In [ ]:
# Analyze a PDF and extract tables with prebuilt-layout
def analyze_pdf_to_tables(pdf_path: str) -> List[pd.DataFrame]:
    with open(pdf_path, 'rb') as f:
        poller = client.begin_analyze_document('prebuilt-layout', document=f)
    result = poller.result()
    tables: List[pd.DataFrame] = []
    for table in result.tables:
        max_col = max([c.column_index for c in table.cells]) if table.cells else 0
        max_row = max([c.row_index for c in table.cells]) if table.cells else 0
        grid = [[None for _ in range(max_col + 1)] for _ in range(max_row + 1)]
        for cell in table.cells:
            grid[cell.row_index][cell.column_index] = cell.content
        df = pd.DataFrame(grid)
        tables.append(df)
    return tables


In [ ]:
# Identify rate cards and surcharges with simple heuristics
def classify_tables(tables: List[pd.DataFrame]):
    rate_rows = []
    surcharge_rows = []
    for df in tables:
        header = ' '.join(str(x).lower() for x in list(df.iloc[0].dropna()))
        if 'zone' in header and ('rate' in header or '$' in header):
            df2 = df.rename(columns=df.iloc[0]).drop(df.index[0]).reset_index(drop=True)
            for _, r in df2.iterrows():
                try:
                    rate_rows.append({
                        'ServiceLevel': None,
                        'Zone': str(r.get('Zone') or r.get('ZONE') or ''),
                        'WeightFrom': None,
                        'WeightTo': None,
                        'Rate': float(str(r.get('Rate') or r.get('RATE') or '0').replace('$','').strip()),
                    })
                except Exception:
                    continue
        elif 'surcharge' in header or 'fee' in header:
            df2 = df.rename(columns=df.iloc[0]).drop(df.index[0]).reset_index(drop=True)
            for _, r in df2.iterrows():
                name = str(r.get('Name') or r.get('Surcharge') or '')
                val = str(r.get('Percent') or r.get('%') or r.get('Amount') or '').replace('%','').replace('$','')
                try:
                    surcharge_rows.append({'Name': name, 'AmountType': 'percent' if '%' in header else 'flat', 'Value': float(val) if val else 0.0, 'MinCharge': None})
                except Exception:
                    continue
    return pd.DataFrame(rate_rows), pd.DataFrame(surcharge_rows)


In [ ]:
# Example: parse a sample PDF from Files and write to Delta tables
pdf_path = 'Files/contracts/sample_carrier.pdf'  # OneLake Files path
carrier_id = 'CARRIER_X'
service_level_default = 'GROUND'

tables = analyze_pdf_to_tables(pdf_path)
rates_df, sur_df = classify_tables(tables)
if len(rates_df) > 0:
    rates_df['CarrierId'] = carrier_id
    rates_df['ServiceLevel'] = rates_df['ServiceLevel'].fillna(service_level_default)
    rates_df['EffectiveDate'] = datetime.utcnow().date()
    spark.createDataFrame(rates_df).write.mode('append').format('delta').saveAsTable('dbo.RateCard')
if len(sur_df) > 0:
    sur_df['CarrierId'] = carrier_id
    sur_df['EffectiveDate'] = datetime.utcnow().date()
    spark.createDataFrame(sur_df).write.mode('append').format('delta').saveAsTable('dbo.Surcharge')
print('Wrote entities to Delta: RateCard, Surcharge')
